# DS3 Datathon 2024 - Celestial Bodies

Description

## Imports

In [ ]:
# Numbers
import pandas as pd
import numpy as np

# Graphs
import seaborn as sns
import matplotlib.pyplot as plt

# ML
import sklearn
import sklearn.model_selection
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
import random

# Pipeline
from sklearn.model_selection import train_test_split # For train/test splits
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline

# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV # For optimization

from sklearn.metrics import balanced_accuracy_score

### Setting final/develpment mode

As it is a datathon submission, we use entirety of the dataset, to generate more accurate results for the final submission.

In [ ]:
generating_final_result = True

### Fixing seeds

[Source](https://sklearn-genetic-opt.readthedocs.io/en/stable/tutorials/reproducibility.html)

In [ ]:
random_seed = 5643
np.random.seed(random_seed)
random.seed(random_seed)

## Loading Data

### Loading training data

In [ ]:
data_train = pd.read_csv("/content/space/celestial_train.csv")

### Separating X values for data_train

In [ ]:
data_trainX = data_train.loc[:,data_train.columns != 'class']

### Separating Y values for data_test

In [ ]:
data_trainY = data_train["class"]

## Data Cleaning
[Source](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

### One hot encoding

In [ ]:
le = LabelEncoder()
le.fit(data_trainY)
data_trainY = le.transform(data_trainY)
data_trainY

array([0, 0, 0, ..., 0, 0, 0])

### Dropping insignificant columns (Determined by feature importance)

In [ ]:
data_trainX = data_trainX.drop(columns=['id', 'fiber_ID', 'cam_col', 'rerun_ID', 'alpha', 'delta', 'run_ID', 'field_ID'])

# Pipeline

In [ ]:
if not generating_final_result:
  X_train, X_test, y_train, y_test = train_test_split(
      data_trainX,
      data_trainY,
      test_size=1/3,
      random_state=0)

  print(X_train.shape)
  print(X_test.shape)
else:
  X_train = data_trainX
  y_train = data_trainY

(33333, 9)
(16667, 9)


In [ ]:
pipe = \
Pipeline(steps=[('scaler', StandardScaler()),
('selector',  SelectFromModel(LinearSVC(C=0.1, penalty="l1", dual=False))),
('classifier', RandomForestClassifier())])

pipe.fit(X_train,y_train)
if not generating_final_result:
  y_pred = pipe.predict(X_test)

print('Training set score: ' + str(pipe.score(X_train,y_train)))
if not generating_final_result:
  print('Test set score: ' + str(pipe.score(X_test,y_test)))

if not generating_final_result:
  print()
  print("Accuracy on test data:", accuracy_score(y_test, y_pred))
  print("Ballanced accuracy on test data:", balanced_accuracy_score(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training set score: 1.0
Test set score: 0.9788204235915282

Accuracy on test data: 0.9788204235915282
Ballanced accuracy on test data: 0.9716476991795141


# Optimization

In [ ]:
parameters = {'scaler': [StandardScaler(), MinMaxScaler(),
              Normalizer(), MaxAbsScaler()],
              'classifier__max_depth': [2,4,6],
              'classifier__min_samples_leaf': [x for x in range(1,10)],
              }

In [ ]:
if not generating_final_result:
  grid = GridSearchCV(pipe, parameters, cv=2).fit(X_train, y_train)

  y_pred = grid.predict(X_test)

  print('Training set score: ' + str(grid.score(X_train, y_train)))
  print('Test set score: ' + str(grid.score(X_test, y_test)))
  print()
  print("Accuracy on test data:", accuracy_score(y_test, y_pred))
  print("Ballanced accuracy on test data:", balanced_accuracy_score(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  w

Training set score: 0.971229712297123
Test set score: 0.9698206035879282

Accuracy on test data: 0.9698206035879282
Ballanced accuracy on test data: 0.9600508025367246


In [ ]:
# Access the best set of parameters
best_params = grid.best_params_
print(best_params)
# Stores the optimum model in best_pipe
best_pipe = grid.best_estimator_
print(best_pipe)

{'classifier__max_depth': 6, 'classifier__min_samples_leaf': 2, 'scaler': MaxAbsScaler()}
Pipeline(steps=[('scaler', MaxAbsScaler()),
                ('selector',
                 SelectFromModel(estimator=LinearSVC(C=0.1, dual=False,
                                                     penalty='l1'))),
                ('classifier',
                 RandomForestClassifier(max_depth=6, min_samples_leaf=2))])


## Creating data for submission

Loading, standarizing and preparing results for the final submission for the datathon

### Loading test data

In [ ]:
data_test = pd.read_csv("/content/space/celestial_test.csv")

### Cleaning test data

Dropping colums determined by feature importance

In [ ]:
y_pred = pipe.predict(data_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- alpha
- cam_col
- delta
- fiber_ID
- field_ID
- ...


In [ ]:
y_pred = le.inverse_transform(y_pred)

### Saving predictions to CSV

In [ ]:
return_dataset = pd.DataFrame({'id': ids, 'output': y_pred}, columns=['id', 'output'])
return_dataset

,id,output
0,50000,QSO
1,50001,GALAXY
2,50002,QSO
3,50003,GALAXY
4,50004,STAR
...,...,...
49995,99995,GALAXY
49996,99996,STAR
49997,99997,QSO
49998,99998,GALAXY


In [ ]:
return_dataset.to_csv("celestial_solutions.csv", index = False)